# Computing the linear momentum

In [1]:
#%matplotlib notebook
import numpy as np
import random as random
import matplotlib.pyplot as plt
import scipy.integrate as integrate
import matplotlib.animation as animation
from scipy.integrate import odeint
from scipy import integrate as intg
from time import time
from scipy.io import loadmat
from scipy.interpolate import interp1d
from tqdm import tqdm
from matplotlib.patches import Rectangle
from matplotlib.backends.backend_pdf import PdfPages
import os
from sklearn.utils import shuffle
import math as math
import pandas as pd
from IPython.display import clear_output
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
#from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d
from itertools import product, combinations
from IPython.display import clear_output

np.random.seed(0)

In [2]:

Vitesses_Dico = np.load('Vitesses_Dico.npy',allow_pickle='TRUE').item()

In [3]:
def extract_dataframe(r,v):
    
    if r==7.5:
        CoordsZ_string= "coords_Z_R"+ str(r)
        vectX_string= "vect_X_R"+ str(r)+"_v"+str(int(v))
        vectY_string= "vect_Y_R"+ str(r)+"_v"+str(int(v))
        vectZ_string= "vect_Z_R"+ str(r)+"_v"+str(int(v))
    else:
        CoordsZ_string= "coords_Z_R"+ str(int(r))
        vectX_string= "vect_X_R"+ str(int(r))+"_v"+str(int(v))
        vectY_string= "vect_Y_R"+ str(int(r))+"_v"+str(int(v))
        vectZ_string= "vect_Z_R"+ str(int(r))+"_v"+str(int(v))
        
        
    A=Vitesses_Dico[CoordsZ_string]
    B=Vitesses_Dico["coords_Y"]
    C=Vitesses_Dico["coords_X"]
    coords= pd.concat([A,B,C], axis=1)
    
    
    Vect_X=Vitesses_Dico[vectX_string]
    Vect_Y=Vitesses_Dico[vectY_string] 
    Vect_Z=Vitesses_Dico[vectZ_string] 
    return(Vect_Z, Vect_Y, Vect_X, coords)


def listes(R):
    if R==7.5:
        liste_axeZ=np.linspace(0.04375,0.05635,nptsZ) 
    elif int(R)==15:
        liste_axeZ=np.linspace(0.04750,0.06010,nptsZ) 
    else:
        liste_axeZ=np.linspace(0.05250,0.06510,nptsZ) 
        
    liste_axeY=np.linspace(0,0.015,nptsY) 
    liste_axeX=np.linspace(0.05,0.054,nptsX)
    return(liste_axeX,liste_axeY,liste_axeZ)

def find_index(R,idxx,idxy,idxz,coords,printval=False):
    if R==7.5:
        liste_axeZ=np.linspace(0.04375,0.05635,nptsZ) 
    elif int(R)==15:
        liste_axeZ=np.linspace(0.04750,0.06010,nptsZ) 
    else:
        liste_axeZ=np.linspace(0.05250,0.06510,nptsZ) 
        
    liste_axeY=np.linspace(0,0.015,nptsY) 
    liste_axeX=np.linspace(0.05,0.054,nptsX)
    
    
    C=coords.loc[(coords['axeY'].round(5) == np.round(liste_axeY[idxy],5)) &(coords['axeX'].round(5) == np.round(liste_axeX[idxx],5)), 'axeZ']
    indexZ=min(range(len(C)), key=lambda i: abs(C.iloc[i]-liste_axeZ[idxz]))
    
    if printval:
        print('R={}, x={}, y={}, z={}'.format(R, liste_axeX[idxx],liste_axeY[idxy],liste_axeZ[idxz]))
    
    return(C.index[indexZ])
    
    
    #return(liste_axeX,liste_axeY,liste_axeZ)

nptsZ,nptsY,nptsX=64,61,41

In [4]:
R=7.5
V=20
vect_Z,vect_Y,vect_X,coords=extract_dataframe(R,V)
#LX,LY,LZ=listes_axes(R)
SmallTab=coords.iloc[::41, :]
C2=SmallTab.index.values.astype(int)

NewTab=coords
NewTab=NewTab.loc[NewTab["axeY"]<=0.005] #5mm et on multiplie par 2

if R==7.5:  
    NewTab=NewTab.loc[NewTab["axeZ"]<=0.05375]
elif R==15:
    NewTab=NewTab.loc[NewTab["axeZ"]<=0.05750]
else:
    NewTab=NewTab.loc[NewTab["axeZ"]<=0.06250]

C3=NewTab.index.values.astype(int)


TX_Sup=NewTab.loc[NewTab["axeX"]==0.05]
TY_Sup=NewTab.loc[NewTab["axeY"]==0]

CX_Sup=TX_Sup.index.values.astype(int)
CY_Sup=TY_Sup.index.values.astype(int)


dx=0.0001
dy=0.00025
dz=0.0002

In [5]:
dx=0.0001
dy=0.00025
dz=0.0002
limtps = 100

for R in [7.5,15,25]:
    for V in [5,10,20,40]:

        vect_Z,vect_Y,vect_X,coords=extract_dataframe(R,V)

        NewTab=coords
        NewTab=NewTab.loc[NewTab["axeY"]<=0.005]

        if R==7.5:  
            NewTab=NewTab.loc[NewTab["axeZ"]<=0.05375]
        elif R==15:
            NewTab=NewTab.loc[NewTab["axeZ"]<=0.05750]
        else:
            NewTab=NewTab.loc[NewTab["axeZ"]<=0.06250]

        C3=NewTab.index.values.astype(int)


        TX_Sup=NewTab.loc[NewTab["axeX"]==0.05]
        TY_Sup=NewTab.loc[NewTab["axeY"]==0]

        CX_Sup=TX_Sup.index.values.astype(int)
        CY_Sup=TY_Sup.index.values.astype(int)


        NORME=pd.DataFrame(np.zeros(limtps),columns=['sum'])
        NXSUP=pd.DataFrame(np.zeros(limtps),columns=['sum'])
        NYSUP=pd.DataFrame(np.zeros(limtps),columns=['sum'])

        #for idx,index in enumerate(C3):
        idx=1
        for idx,index in enumerate(C3): #85731
            if idx%10000==0:
                print(idx)
            test_Z,test_Y,test_X=vect_Z.iloc[[index]].T.reset_index(drop=True),vect_Y.iloc[[index]].T.reset_index(drop=True),vect_X.iloc[[index]].T.reset_index(drop=True)
            P=np.sqrt(test_Z**2+test_Y**2+test_X**2)*dx*dy*dz
            P.rename(columns={P.columns[0]:'sum'}, inplace=True)
            NORME+=P

        for idx,index in enumerate(CX_Sup):   
            test_Z,test_Y,test_X=vect_Z.iloc[[index]].T.reset_index(drop=True),vect_Y.iloc[[index]].T.reset_index(drop=True),vect_X.iloc[[index]].T.reset_index(drop=True)
            P=np.sqrt(test_Z**2+test_Y**2+test_X**2)*dx*dy*dz
            P.rename(columns={P.columns[0]:'sum'}, inplace=True)
            NXSUP+=P

        for idx,index in enumerate(CY_Sup):   
            test_Z,test_Y,test_X=vect_Z.iloc[[index]].T.reset_index(drop=True),vect_Y.iloc[[index]].T.reset_index(drop=True),vect_X.iloc[[index]].T.reset_index(drop=True)
            P=np.sqrt(test_Z**2+test_Y**2+test_X**2)*dx*dy*dz
            P.rename(columns={P.columns[0]:'sum'}, inplace=True)
            NYSUP+=P     

        NORME2=4*NORME-2*NXSUP-2*NYSUP
        titre0='2_Volume_R'+str(R)+'_V'+str(V)+'.csv'
        titre1='2_BackVolume_R'+str(R)+'_V'+str(V)+'.csv'
        
        np.savetxt(titre0,NORME2['sum'].to_numpy())
        np.savetxt(titre1,NORME['sum'].to_numpy())
        clear_output(wait=True)

0
10000
20000
30000
40000
